In [2]:
# Do some imports
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import matlab.engine



# Load data and process data
data_dir = "../data/"

In [2]:
#epsilon = 1
#conn = '8m'
numberimfs = 2
eng = matlab.engine.start_matlab()

In [3]:
def load_data(data_dir):
    train = pd.read_json(data_dir+"test.json")
    # Fill 'na' angles with zero
    train.inc_angle = train.inc_angle.replace('na', 0)
    train.inc_angle = train.inc_angle.astype(float).fillna(0.0)
    return train

train = load_data(data_dir)

In [4]:
# Process data into images
def process_images(df):
    X_band1 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in df["band_1"]])
    X_band2 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in df["band_2"]])
    # Merge bands and add another band as the mean of Band 1 and Band 2 (useful for the ImageDataGenerator later)
    imgs = np.concatenate([X_band1[:, :, :, np.newaxis]
                            , X_band2[:, :, :, np.newaxis]
                            ,((X_band1+X_band2)/2)[:, :, :, np.newaxis]], axis=-1)
    return imgs

X_train = process_images(train)

In [5]:
# Calculate IMFs, residue, and medel(?) using matlab wrapper
def IEMD(img, epsilon=0.1, numberimfs=4, conn='8m'):
    matlab_arr = matlab.double(img.tolist())
    matlab_imfs, matlab_residue, matlab_medel = eng.IEMD_public(matlab_arr, epsilon, numberimfs, conn, nargout=3) 
    
    imfs = np.asarray(matlab_imfs)
    residue = np.asarray(matlab_residue)
    medel = np.asarray(matlab_medel)
    return imfs, residue, medel

In [6]:
# Create the columns in the pandas object for each IMF
for band in range(1,3):
    for imf in range(1,numberimfs + 1):
        column_name = "band_" + str(band) + "_imf_" + str(imf)
        #train[column_name] = np.nan
        train[column_name] = np.empty((len(train), 0)).tolist()
        #train[column_name] = np.zeros((75,75)).reshape(1, 75 * 75).tolist()

In [7]:
# Calculate every IMF for every image
from timeit import default_timer as timer
def IMF_to_JSON(data, name):
    num_img = len(data)
    start_time = timer()
    for i, images in enumerate(data):
        band1_imf, _, _ = IEMD(images[:,:,0], 1, 2)
        band2_imf, _, _ = IEMD(images[:,:,1], 1, 2)

        for imf in range(numberimfs):
            column_name_band1 = "band_1_imf_" + str(imf + 1)
            column_name_band2 = "band_2_imf_" + str(imf + 1)
            train.set_value(i, column_name_band1, band1_imf[:,:,imf].reshape(75 * 75).tolist())
            train.set_value(i, column_name_band2, band2_imf[:,:,imf].reshape(75 * 75).tolist())

        msg = "Image " + str(i) + "/" + str(num_img) + " completed..."
        print (msg, end="\r")
    print("Total run time: " + str(timer() - start_time))
    print("Saving to JSON...")
    train.to_json("../data/imfs_" + name + ".json")
    print("Done!")

In [8]:
IMF_to_JSON(X_train[0:500], "0-500")
IMF_to_JSON(X_train[500:1000], "500-1000")
IMF_to_JSON(X_train[1000:1500], "1000-1500")
IMF_to_JSON(X_train[1500:2000], "1500-2000")
IMF_to_JSON(X_train[2000:2500], "2000-2500")
IMF_to_JSON(X_train[2500:3000], "2500-3000")
IMF_to_JSON(X_train[3000:3500], "3000-3500")
IMF_to_JSON(X_train[3500:4000], "3500-4000")
IMF_to_JSON(X_train[4000:4500], "4000-4500")
IMF_to_JSON(X_train[4500:5000], "4500-5000")
IMF_to_JSON(X_train[5000:5500], "5000-5500")
IMF_to_JSON(X_train[5500:6000], "5500-6000")
IMF_to_JSON(X_train[6000:6500], "6000-6500")
IMF_to_JSON(X_train[6500:7000], "6500-7000")
IMF_to_JSON(X_train[7000:7500], "7000-7500")
IMF_to_JSON(X_train[7500:8000], "7500-8000")
IMF_to_JSON(X_train[8000:], "8000-8500")


Total run time: 938.226117002
Saving to JSON...
Done!
Total run time: 998.3525017459999
Saving to JSON...
Done!
Total run time: 956.4319488759998
Saving to JSON...
Done!
Total run time: 1005.87310282
Saving to JSON...
Done!
Total run time: 981.3900896359996
Saving to JSON...
Done!
Total run time: 934.7283943559996
Saving to JSON...
Done!
Total run time: 952.0812469339999
Saving to JSON...
Done!
Total run time: 973.5494604819996
Saving to JSON...
Done!
Total run time: 956.2692998679995
Saving to JSON...
Done!
Total run time: 994.0282075980012
Saving to JSON...
Done!
Total run time: 971.0275634249992
Saving to JSON...
Done!
Total run time: 1005.8378325379999
Saving to JSON...
Done!
Total run time: 951.8225160149996
Saving to JSON...
Done!
Total run time: 968.480737539001
Saving to JSON...
Done!
Total run time: 991.9668566469991
Saving to JSON...
Done!
Total run time: 967.3648743829999
Saving to JSON...
Done!
Total run time: 822.760027983999
Saving to JSON...
Done!


In [4]:
check1 = pd.read_json(data_dir+"imfs_7500-8000.json")
check1 = check1[check1.astype(str)['band_1_imf_1'] != '[]']
check1.sort_index()
check1.head()

,band_1,band_1_imf_1,band_1_imf_2,band_2,band_2_imf_1,band_2_imf_2,id,inc_angle
0,"[-15.863251, -15.201077, -17.887735, -19.17248...","[-1.4127189213, -0.1207432044, -2.0218206029, ...","[1.1309922293999999, 0.6628434341, 0.079517266...","[-21.629612, -21.142353, -23.908337, -28.34524...","[-0.6148363156000001, 1.7639435407000001, 1.13...","[2.5113698556, 0.9460240326, -0.82980234400000...",5941774d,34.966400
1,"[-26.0589694977, -26.0589694977, -26.058969497...","[-1.625861958, -1.7220543449, -1.774688182, 1....","[-1.3515950490000002, -1.2451483555, -1.155914...","[-25.7542076111, -25.7542076111, -25.754207611...","[-1.9438953228, -2.0361644372, -2.1185283769, ...","[0.375549732, 0.4018488624, 0.3974204925, 0.12...",4023181e,32.615072
10,"[-17.691271, -16.036381, -17.908293, -22.23594...","[-1.7266944402000002, 0.9858551578, 0.26138415...","[2.2806165739, 1.1710349998, 0.0178208555, -1....","[-26.91781, -24.979542, -24.979473, -24.734716...","[1.1252498064, 2.4871679336, 1.8454473811, 1.6...","[-1.453234031, -0.8355782884, -0.1582205243, 0...",565b28ac,35.782900
100,"[-22.6934928894, -16.6730079651, -18.416126251...","[-2.7411529995, 3.1881055312, 1.4841881274, -0...","[0.1563689755, 0.3675269416, 0.447364332800000...","[-20.9147281647, -17.1811561584, -17.833019256...","[-1.244485004, 2.8097429532, 2.5864452151, -1....","[1.33723137, 0.8967400904, 0.3598558502, -0.16...",0bd4658e,41.045646
101,"[-17.6123561859, -17.7215538025, -17.721553802...","[2.8204495955, 2.2880735554, 1.7586233159, 1.3...","[-0.5930168337, -0.2121611838, 0.273743976, 0....","[-14.7677564621, -21.3540878296, -21.354087829...","[3.2505804223, -3.2170211872, -3.0769769905, -...","[0.5976397201, 0.5301652362, 0.4457029878, 0.2...",4a311bb0,46.596700


In [5]:
check2 = pd.read_json(data_dir+"imfs_7000-7500.json")
check2 = check2[check2.astype(str)['band_1_imf_1'] != '[]']
check2.sort_index()
check2.head()

,band_1,band_1_imf_1,band_1_imf_2,band_2,band_2_imf_1,band_2_imf_2,id,inc_angle
0,"[-15.863251, -15.201077, -17.887735, -19.17248...","[-1.4127189213, -0.1207432044, -2.0218206029, ...","[1.1309922293999999, 0.6628434341, 0.079517266...","[-21.629612, -21.142353, -23.908337, -28.34524...","[-0.6148363156000001, 1.7639435407000001, 1.13...","[2.5113698556, 0.9460240326, -0.82980234400000...",5941774d,34.966400
1,"[-26.0589694977, -26.0589694977, -26.058969497...","[-1.625861958, -1.7220543449, -1.774688182, 1....","[-1.3515950490000002, -1.2451483555, -1.155914...","[-25.7542076111, -25.7542076111, -25.754207611...","[-1.9438953228, -2.0361644372, -2.1185283769, ...","[0.375549732, 0.4018488624, 0.3974204925, 0.12...",4023181e,32.615072
10,"[-17.691271, -16.036381, -17.908293, -22.23594...","[-1.7266944402000002, 0.9858551578, 0.26138415...","[2.2806165739, 1.1710349998, 0.0178208555, -1....","[-26.91781, -24.979542, -24.979473, -24.734716...","[1.1252498064, 2.4871679336, 1.8454473811, 1.6...","[-1.453234031, -0.8355782884, -0.1582205243, 0...",565b28ac,35.782900
100,"[-22.6934928894, -16.6730079651, -18.416126251...","[-2.7411529995, 3.1881055312, 1.4841881274, -0...","[0.1563689755, 0.3675269416, 0.447364332800000...","[-20.9147281647, -17.1811561584, -17.833019256...","[-1.244485004, 2.8097429532, 2.5864452151, -1....","[1.33723137, 0.8967400904, 0.3598558502, -0.16...",0bd4658e,41.045646
101,"[-17.6123561859, -17.7215538025, -17.721553802...","[2.8204495955, 2.2880735554, 1.7586233159, 1.3...","[-0.5930168337, -0.2121611838, 0.273743976, 0....","[-14.7677564621, -21.3540878296, -21.354087829...","[3.2505804223, -3.2170211872, -3.0769769905, -...","[0.5976397201, 0.5301652362, 0.4457029878, 0.2...",4a311bb0,46.596700
